In [ ]:
!git clone https://github.com/rahilsinghi/DL_PROJECT_1.git


Cloning into 'DL_PROJECT_1'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 45 (delta 4), reused 7 (delta 3), pack-reused 34 (from 1)
Receiving objects: 100% (45/45), 208.90 MiB | 19.28 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Updating files: 100% (27/27), done.


In [ ]:
!pip install torch torchvision tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [16]:
# cifar_resnet/model.py

import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, initial_planes=32):
        super(ResNet, self).__init__()
        self.in_planes = initial_planes

        # Initial convolution (for CIFAR-10 32x32 images)
        self.conv1 = nn.Conv2d(3, initial_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(initial_planes)

        # Residual layers: we use 4 stages with [2,2,2,2] blocks each
        self.layer1 = self._make_layer(block, initial_planes,   num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, initial_planes*2, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, initial_planes*4, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, initial_planes*8, num_blocks[3], stride=2)

        # Final linear layer
        self.linear = nn.Linear(initial_planes*8*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for s in strides:
            layers.append(block(self.in_planes, planes, s))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))   # (N, initial_planes, 32, 32)
        out = self.layer1(out)                    # (N, initial_planes, 32, 32)
        out = self.layer2(out)                    # (N, initial_planes*2, 16, 16)
        out = self.layer3(out)                    # (N, initial_planes*4, 8, 8)
        out = self.layer4(out)                    # (N, initial_planes*8, 4, 4)
        out = F.avg_pool2d(out, 4)                # (N, initial_planes*8, 1, 1)
        out = out.view(out.size(0), -1)           # (N, initial_planes*8)
        out = self.linear(out)                    # (N, num_classes)
        return out

def ResNet18CIFAR(num_classes=10, initial_planes=32):
    """
    Constructs a ResNet-18 model for CIFAR-10.
    Using initial_planes=32 typically yields a model under 5M parameters.
    """
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes, initial_planes=initial_planes)

In [17]:
#This train file is for model2.py for achieving better than 0.82 (4:51 pm, march 4)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
#from model import ResNet18CIFAR

def train():
    device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Data augmentation & normalization
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.AutoAugment(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])

    transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])

    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    trainloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=4)

    valset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_val)
    valloader = DataLoader(valset, batch_size=128, shuffle=False, num_workers=4)

    model = ResNet18CIFAR().to(device)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

    best_val_acc = 0.0
    for epoch in range(200):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(trainloader, desc=f"Epoch {epoch+1}/200"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        scheduler.step()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in valloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_acc = 100.0 * correct / total
        print(f"Epoch [{epoch+1}/200] Loss: {running_loss/len(trainloader):.4f}, Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print(f"Saved best model at epoch {epoch+1} with Val Acc: {val_acc:.2f}%")
    print("Training complete. Best Accuracy: {:.2f}%".format(best_val_acc))

if __name__ == "__main__":
    train()

Using device: cuda


100%|██████████| 170M/170M [00:13<00:00, 12.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Epoch 1/200: 100%|██████████| 196/196 [00:40<00:00,  4.80it/s]


Epoch [1/200] Loss: 2.1203, Val Acc: 33.89%
Saved best model at epoch 1 with Val Acc: 33.89%


...

Epoch [198/200] Loss: 0.5614, Val Acc: 94.53%
Saved best model at epoch 198 with Val Acc: 94.53%


Epoch 199/200: 100%|██████████| 196/196 [00:38<00:00,  5.15it/s]


Epoch [199/200] Loss: 0.5622, Val Acc: 94.45%


Epoch 200/200: 100%|██████████| 196/196 [00:38<00:00,  5.03it/s]


Epoch [200/200] Loss: 0.5620, Val Acc: 94.51%
Training complete. Best Accuracy: 94.53%


In [23]:
# cifar_resnet/inference.py

import os
import pickle
import numpy as np
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
#from model import ResNet18CIFAR

def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

class CIFARTestDataset(Dataset):
    def __init__(self, data, ids, transform=None):
        """
        data: numpy array of shape (N, 32, 32, 3)
        ids: array-like of test image IDs
        transform: transforms to apply
        """
        self.data = data
        self.ids = ids
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # If data is stored as (32,32,3)
        img = self.data[idx].astype("uint8")
        img_id = self.ids[idx]
        if self.transform:
            img = self.transform(img)
        return img, img_id

def main():
    # Device selection: CUDA > MPS > CPU
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
    print("Using device:", device)

    # Load the best model (make sure parameters match training)
    model = ResNet18CIFAR(num_classes=10, initial_planes=32).to(device)
    model.load_state_dict(torch.load("best_model.pth", map_location=device))
    model.eval()

    # Load custom test set (provided by competition) – adjust path as needed
    # The custom test file should contain keys: b'data' (shape (N, 32,32,3)) and b'ids'
    test_file = "/content/cifar_test_nolabel.pkl"
    test_dict = unpickle(test_file)
    print("Keys in test_dict:", test_dict.keys())

    test_images = test_dict[b'data']  # shape (N, 32, 32, 3)
    test_ids = test_dict[b'ids']

    # Define test transforms (no random augmentation)
    transform_test = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2470, 0.2435, 0.2616))
    ])

    test_dataset = CIFARTestDataset(test_images, test_ids, transform=transform_test)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

    # Inference loop
    all_preds = []
    all_ids = []
    with torch.no_grad():
        for images, ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy().tolist())
            all_ids.extend(ids.tolist())

    # Create submission CSV (must have exactly two columns: "ID" and "Labels")
    submission_df = pd.DataFrame({
        "ID": all_ids,
        "Labels": all_preds
    })
    submission_df.to_csv("submission.csv", index=False)
    print("submission.csv generated! Ready for Kaggle submission.")

if __name__ == "__main__":
    main()

Using device: cuda
Keys in test_dict: dict_keys([b'data', b'ids'])


<ipython-input-23-bd3453630746>:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth", map_location=device))


submission.csv generated! Ready for Kaggle submission.


Do't touch anything beyond this. The followig cells got us 0.82861. Do all edits and chnages above this.

In [ ]:
#model.py and train.py
import os
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm

 # Ensure this function is at the module level in model.py

def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

class CIFARDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data  # shape (N, 3072)
        self.labels = labels  # shape (N,)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Reshape vector (3072,) -> (32, 32, 3), then convert to PIL Image
        img = self.data[idx].reshape(3, 32, 32).transpose(1, 2, 0).astype("uint8")
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

def load_cifar10_batches(root_dir):
    data_list, labels_list = [], []
    for i in range(1, 6):
        batch_file = os.path.join(root_dir, f"data_batch_{i}")
        batch = unpickle(batch_file)
        data_list.append(batch[b'data'])
        labels_list.extend(batch[b'labels'])
    X = np.concatenate(data_list, axis=0)
    y = np.array(labels_list)
    return X, y

def main():
    # Set up device: if MPS is available on your MacBook Pro, use it.

    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"
    print("Using device:", device)


    # Load CIFAR-10 training data
    root_dir = "/content/DL_PROJECT_1/cifar-10-python/cifar-10-batches-py"  # adjust path if needed
    X, y = load_cifar10_batches(root_dir)

    # Split into training and validation sets (90/10 split)
    num_samples = len(X)
    split_idx = int(num_samples * 0.9)
    X_train, X_val = X[:split_idx], X[split_idx:]
    y_train, y_val = y[:split_idx], y[split_idx:]

    # Define data augmentation and normalization for training; only normalization for validation
    transform_train = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        # Uncomment the following line to try color jitter (optional)
        # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    transform_val = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])

    train_dataset = CIFARDataset(X_train, y_train, transform=transform_train)
    val_dataset = CIFARDataset(X_val, y_val, transform=transform_val)

    # Increase num_workers if your system can support it; pin_memory only if not on MPS.
    train_loader = DataLoader(
    train_dataset,
    batch_size=256,         # larger batch size
    shuffle=True,
    num_workers=2,          # or 8 if your CPU can handle it
    pin_memory=False        # MPS typically doesn’t benefit
)
    val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=False)

    # Instantiate the model; use the same architecture as during training.
    # For example, if you trained with 64 initial channels, pass initial_planes=64.
    model = ResNet18CIFAR(num_classes=10, initial_planes=32).to(device)

    # Define loss function and optimizer (SGD with momentum and weight decay is standard for CIFAR-10)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    # Use cosine annealing for a smooth LR schedule (T_max set to number of epochs)
    num_epochs = 200
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    best_val_acc = 0.0

    # Optionally, for CUDA you can use AMP (MPS support is experimental)
    use_amp = (device == "cuda")
    scaler = torch.cuda.amp.GradScaler() if use_amp else None

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        train_progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
        for images, labels in train_progress:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            if use_amp:
                with torch.cuda.amp.autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            running_loss += loss.item()
            train_progress.set_postfix(loss=loss.item())

        scheduler.step()

        # Evaluate on validation set
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100.0 * correct / total
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {avg_loss:.4f} | Val Acc: {val_acc:.2f}%")

        # Save best model checkpoint
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print(f"Saved new best model at epoch {epoch+1} with Val Acc: {val_acc:.2f}%")

    print("Training complete.")
    print(f"Best Validation Accuracy: {best_val_acc:.2f}%")

if __name__ == "__main__":
    main()


Using device: cuda


<ipython-input-21-7eeb3e5d2b76>:112: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if use_amp else None
Epoch 1/200:   0%|          | 0/176 [00:00<?, ?it/s]<ipython-input-21-7eeb3e5d2b76>:124: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch [1/200] - Loss: 1.7395 | Val Acc: 47.20%
Saved new best model at epoch 1 with Val Acc: 47.20%


Epoch [2/200] - Loss: 1.3147 | Val Acc: 55.80%
Saved new best model at epoch 2 with Val Acc: 55.80%


Epoch [3/200] - Loss: 1.0314 | Val Acc: 67.44%
Saved new best model at epoch 3 with Val Acc: 67.44%


...

Epoch [196/200] - Loss: 0.0018 | Val Acc: 94.50%


Epoch [197/200] - Loss: 0.0018 | Val Acc: 94.46%


Epoch [198/200] - Loss: 0.0017 | Val Acc: 94.40%


Epoch [199/200] - Loss: 0.0018 | Val Acc: 94.40%


Epoch [200/200] - Loss: 0.0018 | Val Acc: 94.48%
Training complete.
Best Validation Accuracy: 94.62%


In [ ]:
# inference.py
import os
import pickle
import numpy as np
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
# test_file = "/Users/rahilsinghi/Desktop/DL_PROJECT_1/data/cifar_test_nolabel.pkl"

# test_dict = unpickle(test_file)
# print("Keys in test_dict:", test_dict.keys())

class CIFARTestDataset(Dataset):
    def __init__(self, data, ids, transform=None):
        """
        data: numpy array of shape (N, 3072)
        ids: list or numpy array of shape (N,)
        transform: torchvision transform to apply
        """
        self.data = data
        self.ids = ids
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Convert 3072 vector -> (3, 32, 32) -> (32, 32, 3)
        img = self.data[idx].reshape(3, 32, 32).transpose(1, 2, 0).astype("uint8")
        img_id = self.ids[idx]

        if self.transform:
            img = self.transform(img)

        return img, img_id

def main():
    # 1) Load best model
    if torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"
    print(f"Using device: {device}")

    # Instantiate your model and move it to the device
    model = ResNet18CIFAR(num_classes=10, initial_planes=32).to(device)
    model.load_state_dict(torch.load("best_model.pth", map_location=device))
    model.eval()


    # 2) Load the custom test set
    #   Path is relative to the script location (cifar_resnet/).
    #   Adjust if your file is somewhere else.
    test_file = "/content/cifar_test_nolabel.pkl"
    test_dict = unpickle(test_file)
    # Typically, test_dict might have keys like b'images' and b'ids'
    test_images = test_dict[b'data']  # shape (N, 3072)
    test_ids = test_dict[b'ids']        # shape (N,)

    # 3) Define transforms
    transform_test = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2470, 0.2435, 0.2616))
    ])

    # 4) Create Dataset and DataLoader
    test_dataset = CIFARTestDataset(test_images, test_ids, transform=transform_test)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

    # 5) Run inference and collect predictions
    all_preds = []
    all_ids = []
    with torch.no_grad():
        for images, ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy().tolist())
            all_ids.extend(ids.numpy().tolist())  # or just .tolist()

    # 6) Create submission.csv
    submission_df = pd.DataFrame({
        "ID": all_ids,
        "Labels": all_preds
    })
    submission_df.to_csv("submission.csv", index=False)
    print("submission.csv generated! You can submit this file to Kaggle.")

if __name__ == "__main__":
    main()
